# Plan

## Import libraries

In [1]:
import os

import numpy as np
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from shutil import copyfile
from xlsxwriter.utility import xl_col_to_name

import warnings
warnings.filterwarnings("ignore")

## Set ups

In [2]:
# Get the path to the working directory
wd = os.getcwd()

## Harvest the data and task

In [3]:
# Import the data as a DataFrame
plan = pd.read_excel(wd + '/data/plan.xlsx', header=None, skiprows=2, na_values=[' ', '-'])
# Convert a spreadsheet number to its column letter
header = plan.columns.tolist()
plan.columns = [xl_col_to_name(x) for x in header]
# Drop anonymous participants
plan.dropna(subset=['I', 'J'], inplace=True)
plan.reset_index(drop=True, inplace=True)
# Add Full name column
plan['I'] = plan['I'].astype('str')
plan['J'] = plan['J'].astype('str')
plan['Full name'] = plan['I'] + ' ' + plan['J']
plan = plan[plan['Full name'].map(len) > 10]

# Import the task as a ExcelFile
task = pd.ExcelFile(wd + '/templates/template_plan.xlsx')

## Modify the data

In [4]:
# Decode the data using custom mappers
with open(wd +'/data/' + 'subdivisions.txt', 'r') as txt_file:
    for line in txt_file:
        subdivisions = line.strip('[').strip(']').replace("'", "").split(',')
subdivision_decoder = dict(zip([x for x in range(1, 32) if x != 10], subdivisions))
plan['K'] = plan['K'].map(subdivision_decoder)      

with open(wd +'/data/' + 'occupation.txt', 'r') as txt_file:
    for line in txt_file:
        occupation = line.strip('[').strip(']').replace("'", "").split(',')
occupation_decoder = dict(zip([x for x in range(1, 16)] + [97], occupation))
plan['L'] = plan['L'].map(occupation_decoder)

rx_decoder = {1: 'Проводить очные консультации обучающимся (индивидуальные и групповые) вне рамок дисциплин'}
ry_decoder = {1: 'Читать и рецензировать письменные работы обучающихся, включая ВКР и диссертационные работы, участвовать в защитах ВКР и диссертационных работ'}
rz_decoder = {1: 'Проводить разные виды учебных и производственных практик, школ, экспедиций'}
sb_decoder = {1: 'Не планирую выполнять иную образовательную деятельность, связанную с преподаванием дисциплин и научным руководством'}
sc_decoder = {1: 'Не планирую выполнять иную образовательную деятельность'}

plan['RX'] = plan['RX'].map(rx_decoder)
plan['RY'] = plan['RY'].map(ry_decoder)
plan['RZ'] = plan['RZ'].map(rz_decoder)
plan['SB'] = plan['SB'].map(sb_decoder)
plan['SC'] = plan['SC'].map(sc_decoder)

origin = pd.read_excel(wd + '/data/plan.xlsx', header=None)
header = origin.columns
origin.columns = [xl_col_to_name(x) for x in header]
columns = ['LM', 'LN', 'LO', 'LP', 'LQ', 'LR',
           'LS', 'LT', 'LU', 'LV', 'LW', 'LX',
           'LY', 'LZ', 'MA', 'MB', 'MC', 'MD',
           'ME', 'MF', 'MG', 'MH', 'MI', 'MJ',
           'MK', 'ML', 'MN', 'MO', 'MP',
           'MQ', 'MR', 'MS', 'MT', 'MU', 'MV',
           'MW', 'MX', 'MY', 'MZ', 'NA', 'NB',
           'NC', 'ND', 'NE', 'NF', 'NG', 'NH',
           'NI', 'NK', 'NL', 'NM', 'NN',
           'NO', 'NQ', 'NS', 'NT', 'NU',
           'NV', 'NW', 'NX', 'NY', 'NZ', 'OA',
           'OB', 'OC', 'OE', 'OF', 'OG']
origin = origin.loc[:, columns]
mapper = dict(origin.iloc[1])
for k, v in mapper.items():
    mapper[k] = str(v.split(' - ')[-1])

for column in columns:
    if not column in ['MN', 'NK', 'NQ', 'NS', 'OE', 'OF', 'OG']:
        plan[column] = plan[column].map({1: mapper[column]}) 
    else:
        continue

plan['LSOG'] = plan['LS'] + ': ' + plan['OG']

## Parse the data

In [5]:
sheets_to_consider = ['1.1.', '1.2.', '1.3.', '1.4.', '1.5.',
                      '2', '3', '4',
                      '5.1', '5.2', '5.3.', '5.4', '5.5.',
                      '6.1.', '6.2.', '6.3.', '6.4.', '6.5.', '6.6.', '6.7.',
                      '7.1.', '7.2.', '7.3.',
                      '8', '10', '11']

for subdivision in subdivisions:
    file_name = wd + '/results/plan/' + subdivision + '_план_2023' + '.xlsx'
    # Copy the teplate for each subdivision
    copyfile(wd + '/templates/' + 'template_plan_final.xlsx', file_name)
    # Load the file
    workbook = load_workbook(file_name)
    # Select observations
    subset = plan[plan['K'] == subdivision]
    
    for sheet_name in sheets_to_consider:
        print(sheet_name)
        # Parse a particular sheet and save it as a DataFrame
        sheet = task.parse(sheet_name=sheet_name, skiprows=1)
        # Drop NaNs
        sheet.dropna(how='all', inplace=True)
        # Drop useless column
        sheet.drop(columns='№', inplace=True)
        # Save headers
        header = sheet.columns
        
        # Initialize a list for DataFrames
        dfs = []
        
        # Iterate over each row
        for i, data in sheet.iterrows():
            values = data.values.tolist()
            if sheet_name == '4':
                if pd.Series(values).isna().sum() >= 1:
                    values.pop()
                    values.extend(['NA'])
                # Select needed columns and drop missing values
                df = subset.loc[:, values].dropna(thresh=1)
                # Use initial column names
                df.columns = header
                # Append the obtained DataFrame to the list of DataFrames
                dfs.append(df)
                
            elif sheet_name in ['2', '3', '6.4.', '6.5.', '6.6.', '6.7.', '7.3.', '10', '11']:
                # Select needed columns and drop rows with less than 1 non-missing value
                df = subset.loc[:, data.values].dropna(thresh=1)
                # Use initial column names
                df.columns = header
                # Append the obtained DataFrame to the list of DataFrames
                dfs.append(df)
            else:
                # Select needed columns and drop rows with less than 2 non-missing value
                df = subset.loc[:, data.values].dropna(thresh=2)
                # Use initial column names
                df.columns = header
                # Append the obtained DataFrame to the list of DataFrames
                dfs.append(df)

        # Concat the DataFrames
        dfs = pd.concat(dfs, ignore_index=True)
        # Save index as a column
        dfs.reset_index(inplace=True)
        # Change index so that it starts with 1
        dfs['index'] = dfs['index'] + 1
        
        # Get access to the particular sheet
        worksheet = workbook[sheet_name]
        # Turn the DataFrame into Generator object dataframe_to_rows
        rows = dataframe_to_rows(dfs, index=False, header=False)
        # Write data directly in the current Excel sheet
        for r_idx, row in enumerate(rows, 3):
            for c_idx, value in enumerate(row, 1):
                worksheet.cell(row=r_idx, column=c_idx, value=value)
    # Save the updated Excel file
    workbook.save(file_name)

1.1.
1.2.
1.3.
1.4.
1.5.
2
3
4
5.1
5.2
5.3.
5.4
5.5.
6.1.
6.2.
6.3.
6.4.
6.5.
6.6.
6.7.
7.1.
7.2.
7.3.
8
10
11
1.1.
1.2.
1.3.
1.4.
1.5.
2
3
4
5.1
5.2
5.3.
5.4
5.5.
6.1.
6.2.
6.3.
6.4.
6.5.
6.6.
6.7.
7.1.
7.2.
7.3.
8
10
11
1.1.
1.2.
1.3.
1.4.
1.5.
2
3
4
5.1
5.2
5.3.
5.4
5.5.
6.1.
6.2.
6.3.
6.4.
6.5.
6.6.
6.7.
7.1.
7.2.
7.3.
8
10
11
1.1.
1.2.
1.3.
1.4.
1.5.
2
3
4
5.1
5.2
5.3.
5.4
5.5.
6.1.
6.2.
6.3.
6.4.
6.5.
6.6.
6.7.
7.1.
7.2.
7.3.
8
10
11
1.1.
1.2.
1.3.
1.4.
1.5.
2
3
4
5.1
5.2
5.3.
5.4
5.5.
6.1.
6.2.
6.3.
6.4.
6.5.
6.6.
6.7.
7.1.
7.2.
7.3.
8
10
11
1.1.
1.2.
1.3.
1.4.
1.5.
2
3
4
5.1
5.2
5.3.
5.4
5.5.
6.1.
6.2.
6.3.
6.4.
6.5.
6.6.
6.7.
7.1.
7.2.
7.3.
8
10
11
1.1.
1.2.
1.3.
1.4.
1.5.
2
3
4
5.1
5.2
5.3.
5.4
5.5.
6.1.
6.2.
6.3.
6.4.
6.5.
6.6.
6.7.
7.1.
7.2.
7.3.
8
10
11
1.1.
1.2.
1.3.
1.4.
1.5.
2
3
4
5.1
5.2
5.3.
5.4
5.5.
6.1.
6.2.
6.3.
6.4.
6.5.
6.6.
6.7.
7.1.
7.2.
7.3.
8
10
11
1.1.
1.2.
1.3.
1.4.
1.5.
2
3
4
5.1
5.2
5.3.
5.4
5.5.
6.1.
6.2.
6.3.
6.4.
6.5.
6.6.
6.7.
7.1.
7.2.
7.3.
8
10
11
1

# Report

## Import libraries

In [21]:
import os

import numpy as np
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from shutil import copyfile
from xlsxwriter.utility import xl_col_to_name

import warnings
warnings.filterwarnings("ignore")

## Set ups

In [22]:
# Get the path to the working directory
wd = os.getcwd()

## Harvest the data and task

In [29]:
# Import the data as a DataFrame
report = pd.read_excel(wd + '/data/report.xlsx', header=None, skiprows=2, na_values=[' ', '-'])
# Convert a spreadsheet number to its column letter
header = report.columns.tolist()
report.columns = [xl_col_to_name(x) for x in header]
# Drop anonymous participants
report.dropna(subset=['I', 'J'], inplace=True)
report.reset_index(drop=True, inplace=True)
# Add Full name column
report['I'] = report['I'].astype('str')
report['J'] = report['J'].astype('str')
report['Full name'] = report['I'] + ' ' + report['J']
report = report[report['Full name'].map(len) > 10]

# Import the task as a ExcelFile
task = pd.ExcelFile(wd + '/templates/template_report.xlsx')

## Modify the data

In [30]:
# Decode the data using custom mappers
with open(wd +'/data/' + 'subdivisions.txt', 'r') as txt_file:
    for line in txt_file:
        subdivisions = line.strip('[').strip(']').replace("'", "").split(',')
subdivision_decoder = dict(zip([x for x in range(1, 32) if x != 10], subdivisions))
report['K'] = report['K'].map(subdivision_decoder)      

with open(wd +'/data/' + 'occupation.txt', 'r') as txt_file:
    for line in txt_file:
        occupation = line.strip('[').strip(']').replace("'", "").split(',')
occupation_decoder = dict(zip([x for x in range(1, 16)] + [97], occupation))
report['L'] = report['L'].map(occupation_decoder)

po_decoder = {1: 'Проводил(а) очные консультации обучающимся (индивидуальные и групповые) вне рамок дисциплин'}
pp_decoder = {1: 'Читал(а) и рецензировал(а) письменные работы обучающихся, включая ВКР и диссертационные работы, участвовал(а) в защитах ВКР и диссертационных работ'}
pq_decoder = {1: 'Проводил(а) разные виды учебных и производственных практик, школ, экспедиций'}
ps_decoder = {1: 'Нет'}
pt_decoder = {1: 'Не выполнял(а) иную образовательную деятельность'}

report['PO'] = report['PO'].map(po_decoder)
report['PP'] = report['PP'].map(pp_decoder)
report['PQ'] = report['PQ'].map(pq_decoder)
report['PS'] = report['PS'].map(ps_decoder)
report['PT'] = report['PT'].map(pt_decoder)

origin = pd.read_excel(wd + '/data/report.xlsx', header=None)
header = origin.columns
origin.columns = [xl_col_to_name(x) for x in header]
columns = ['JC', 'JD', 'JE', 'JF', 'JG', 'JH',
           'JI', 'JJ', 'JK', 'JL', 'JM', 'JN',
           'JO', 'JP', 'JQ', 'JR', 'JS', 'JT',
           'JU', 'JV', 'JW', 'JX', 'JY', 'JZ',
           'KA', 'KB', 'KC', 'KE', 'KF',
           'KG', 'KH', 'KI', 'KJ', 'KK', 'KL',
           'KM', 'KN', 'KO', 'KP', 'KQ', 'KR',
           'KS', 'KT', 'KU', 'KV', 'KW', 'KX',
           'KY', 'KZ', 'LB', 'LC', 'LD',
           'LF', 'LE', 'LI', 'LJ',
           'LK', 'LL', 'LM', 'LN', 'LO', 'LP',
           'LQ', 'LR', 'LS', 'LT', 'LV',
           'LW', 'LX']
origin = origin.loc[:, columns]
mapper = dict(origin.iloc[1])

for k, v in mapper.items():
    mapper[k] = str(v.split(' - ')[-1])

mapper['LW'] = 'Работа в интересах НИУ ВШЭ и другие внешние организации и ведомства (привидите краткое описание деятельности/проекта, его направление и результаты): '
mapper['LX'] = 'Работа в интересах НИУ ВШЭ и другие внешние организации и ведомства (привидите краткое описание деятельности/проекта, его направление и результаты): '

for column in columns:
    if not column in ['KE', 'LB', 'LH', 'LI', 'LV', 'LW', 'LX', 'LJ']:
        report[column] = report[column].map({1: mapper[column]}) 
    else:
        continue
        
report['LELH'] = report['LE'] + ': ' + report['LH']
report['LX'] = mapper['LX'] + report['LX']
report['LW'] = mapper['LW'] + report['LW']

## Parse the data

In [32]:
sheets_to_consider = ['1.1.', '1.2.', '1.3.', '1.4.', '1.5.',
                      '2', '3',
                      '4.1.', '4.2.', '4.3.', '4.4.', '4.5.',
                      '5.1.', '5.2.', '5.3.', '5.4.', '5.5.', '5.6.',
                      '6.1.1.', '6.1.2.', '6.1.3.', '6.1.4.', '6.1.5.',
                      '6.2.', '6.3.',
                      '7.', '7.1.', '7.2.',
                      '8', '9', '10']

for subdivision in subdivisions:
    file_name = wd + '/results/report/' + subdivision + '_отчет_2022' + '.xlsx'
    # Copy the teplate for each subdivision
    copyfile(wd + '/templates/' + 'template_report_final.xlsx', file_name)
    # Load the file
    workbook = load_workbook(file_name)
    # Select observations
    subset = report[report['K'] == subdivision]

    for sheet_name in sheets_to_consider:
        # Parse a particular sheet and save it as a DataFrame
        sheet = task.parse(sheet_name=sheet_name, skiprows=1)
        # Drop NaNs
        sheet.dropna(how='all', inplace=True)
        # Drop useless column
        sheet.drop(columns='№', inplace=True)
        # Save headers
        header = sheet.columns

        # Initialize a list for DataFrames
        dfs = []

        # Iterate over each row
        for i, data in sheet.iterrows():
            values = data.values.tolist()
            if sheet_name == '4.1.':
                if pd.Series(values).isna().sum() >= 1:
                    values.pop()
                    values.extend(['NA'])
                # Select needed columns and drop missing values
                df = subset.loc[:, values].dropna(thresh=2)
                # Use initial column names
                df.columns = header
                # Append the obtained DataFrame to the list of DataFrames
                dfs.append(df)
            elif sheet_name in ['1.1.', '1.2.', '1.3.', '1.4.', '1.5.',
                                '5.2.', '5.4.', '5.5.', '5.6.',
                                '6.1.1.', '6.1.2.', '6.1.3.', '6.1.4.', '6.1.5.',
                                '6.2.', '6.3.',
                                '7.2.',
                                '8', '9', '10']:
                # Select needed columns and drop rows with less than 1 non-missing value
                df = subset.loc[:, data.values].dropna(thresh=1)
                # Use initial column names
                df.columns = header
                # Append the obtained DataFrame to the list of DataFrames
                dfs.append(df) 
            else:
                # Select needed columns and drop rows with less than 2 non-missing value
                df = subset.loc[:, data.values].dropna(thresh=2)
                # Use initial column names
                df.columns = header
                # Append the obtained DataFrame to the list of DataFrames
                dfs.append(df)

        # Concat the DataFrames
        dfs = pd.concat(dfs, ignore_index=True)
        # Save index as a column
        dfs.reset_index(inplace=True)
        # Change index so that it starts with 1
        dfs['index'] = dfs['index'] + 1
        
        # Get access to the particular sheet
        worksheet = workbook[sheet_name]
        # Turn the DataFrame into Generator object dataframe_to_rows
        rows = dataframe_to_rows(dfs, index=False, header=False)
        # Write data directly in the current Excel sheet
        for r_idx, row in enumerate(rows, 3):
            for c_idx, value in enumerate(row, 1):
                worksheet.cell(row=r_idx, column=c_idx, value=value)
    # Save the updated Excel file
    workbook.save(file_name)